## CNN-RNN multi-label classification

This notebook contains the preprocessing that was necessary to use the CNN-RNN classification repository 
(https://github.com/AmrMaghraby/CNN-RNN-A-Unified-Framework-for-Multi-label-Image-Classification)

Furthermore, it contains the command line to use from terminal to run the downloaded code, in conjuction with the data_loader.py that was written to work specifically with my data.

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import json
from pathlib import Path

#### MyData.json

In [2]:
with open('../data/filenames.pkl', 'rb') as infile:
    filenames = pickle.load(infile)
    
with open('../data/labels.pkl', 'rb') as infile2:
    labels = pickle.load(infile2)
    
df = pd.concat([pd.Series(filenames, name='filenames'), pd.Series(labels, name='labels')], axis=1)

labels_dict = {0:'people',1:'objects',2:'places',3:'architecture',4:'abstraction',5:'society',\
          6:'nature',7:'emotions, concepts and ideas',8:'interiors',9:'work and occupations', \
          10:'symbols & personifications',11:'religion and belief',12:'leisure and pastimes',\
          13:'history',14:'literature and fiction',15:'group/movement'}

labels_names = [[]]*len(df.labels)
for k in range(len(df.labels)):
    labels_names[k] = []
    for i in range(len(df.labels[k])):
        if df.labels[k][i] == 1:
            labels_names[k].append(labels_dict[i])

print(len(labels_names))
out = [(filenames[i].split('\\')[-1], labels_names[i]) for i in range(len(labels_names))]
import pickle
import json
from pathlib import Path

with open('../data/results/myData.json', 'w') as outF:
    aux = {}
    for line in out:
        if "group/movement" in line[1]:
            continue
        aux[line[0]] = line[1]
    json.dump(aux, outF)

24999


#### Idx2path.json

In [3]:
d = json.load(open("../data/results/MyData.json"))
words = {}
for k in d.keys():
    for word in d[k]:
        if word not in words.keys():
            words[word] = 0
        words[word] += 1
print(len(words.keys()))

for k in d.keys():
    print(k, d[k])
    break
    
idx2path = {}

for i, k in enumerate(d.keys()):
    idx2path[str(i)] = k

for k in idx2path.keys():
    print(k, idx2path[k])
    break
    
json.dump(idx2path, open("../data/results/idx2path.json", 'w'))

15
A00001_8.jpg ['people', 'religion and belief']
0 A00001_8.jpg


#### zh_vocab.pkl

In [ ]:
lista = []
for word in words.keys():
    print(word, words[word])
    lista.append(word)
print(lista)
json.dump(lista, open("../data/results/MyWords.json", 'w'))

In [5]:
! python CNN_RNN/build_vocab.py

vocab dict_keys(['<pad>', '<start>', '<end>', '<unk>', 'people', 'religion and belief', 'emotions, concepts and ideas', 'work and occupations', 'society', 'nature', 'literature and fiction', 'objects', 'places', 'architecture', 'symbols & personifications', 'leisure and pastimes', 'interiors', 'abstraction', 'history'])
Total vocabulary size: 19
Saved the vocabulary wrapper to '../data/results/zh_vocab.pkl'


### Running the code from terminal:

In [ ]:
! python CNN_RNN/train.py --vocab_path ../data/zh_vocab.pkl --caption_path ../data/MyData.json --image_dir ../../data_tate --idx2path_path ../data/idx2path.json